# Modelagem e Simulação de Canal no OAI (OpenAirInterface)

### 1. O que é modelagem de canal?



Modelos de canal simulam os efeitos do meio físico na transmissão sem fio. Eles consideram:

- **Atenuação (pathloss)**;
- **Interferência**;
- **Desvanecimento (fading)**;
- **Atrasos (delay spread)**.

Esses modelos permitem simular cenários como:
- Áreas urbanas densas,
- Ambientes internos,
- Zonas rurais.

Com isso, é possível avaliar o desempenho da rede sob diferentes condições reais.





### 2. Simulação de Canal com RFsimulator (OAI)


O OpenAirInterface (OAI) possui o módulo **RFsimulator**, que permite:

- Aplicar modelos de canal (ex: AWGN, Rayleigh, Rician);
- Alterar parâmetros em tempo de execução;
- Simular sem a necessidade de rádio físico (SDR).

**obs:** esse tutorial foi feito com base em https://gitlab.eurecom.fr/oai/openairinterface5g/-/blob/develop/openair1/SIMULATION/TOOLS/DOC/channel_simulation.md#channel-modeling


A definição, configuração e modificação em tempo real de um modelo de canal são implementadas em um código comum. Este código está incluído no UE, gNB e eNB. É utilizado ao operar com o RFSimulator. Os simuladores PHY também empregam simulação de canal, mas sua configuração é realizada por meio de opções de linha de comando dedicadas.


### 3. Configuração de Modelo de Canal

Para configurar os efeitos do canal no OAI precisamos adicionar as informações das configurações nos arquivos de configuração da gNB e do UE.

Neste tutorial iremos utilizar os arquivos:
- **Arquivo de configuração do UE:** ../targets/PROJECTS/GENERIC-NR-5GC/CONF/ue.conf
- **Arquivo de configuração da gNB monolito:** ../targets/PROJECTS/GENERIC-NR-5GC/CONF/gnb.sa.band78.fr1.106PRB.usrpb210.conf
- **Arquivo de configuração do canal:** "/home/dco1020/openairinterface5g/ci-scripts/conf_files/channelmod_rfsimu.conf"



Para usar um modelo de canal personalizado com `RFsimulator`, é necessário garantir que os arquivos de configuração do **gNB**, **UE** e o **channelmod** estejam na **mesma pasta**.



**Passo 3.1:** Copiar o arquivo `channelmod_rfsimu.conf` para o diretório de configuração

```bash
cp /home/dco1020/openairinterface5g/ci-scripts/conf_files/channelmod_rfsimu.conf \
/home/dco1020/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/
```
**Passo 3.2:** Verifique se o arquivo foi copiado corretamente com os comandos:

```bash
cd /home/dco1020/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/
ls
```
Veja uma tela similar a :
![passo2](FIG/passo2.jpeg)

O arquivo channelmod_rfsimu.conf deve aparecer listado no terminal.

**Passo 3.3:** Para definir e usar um modelo de canal para uplink, o arquivo de configuração do gNB precisa incluir um arquivo de configuração de canal. Para isso, adicione: 
```bash
@include "channelmod_rfsimu.conf" 
```
ao final do arquivo da gNB.

Faça digitando em um terminal:

```bash
gedit /home/dco1020/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/gnb.sa.band78.fr1.106PRB.usrpb210.conf
```
O arquivo deve ficar similar a:
![passo3](FIG/passo3.jpeg)

**Passo 3.4:** O mesmo deve ser feito para o downlink, incluindo o arquivo de configuração do modelo de canal no final do arquivo de configuração do UE.

Faça digitando em um terminal:

```bash
gedit /home/dco1020/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/ue.conf
```
O arquivo deve ficar similar a:
![passo4](FIG/passo4.jpeg)
**Passo 3.5:** Para alterar parâmetros do modelo de canal é preciso editar o `channelmod_rfsimu.conf`.

Faça digitando em um terminal:

```bash
gedit /home/dco1020/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/channelmod_rfsimu.conf
```
Edite os parâmetros desejados tanto no `rfsimu_channel_enB0` quanto no `rfsimu_channel_ue0`.

![passo5](FIG/passo5.jpeg)

### 4. Execute o OAI com um modelo de canal



**Passo 4.1:** Para habilitar o simulador de canal de RF com funcionalidade de modelagem, adicione a seguinte opção à linha de comando ao iniciar o `gNB` e o `UE`:

```bash
--rfsimulator.options chanmod
```

Para especificar qual modelo de canal deve ser usado, indique o nome da lista com a seguinte opção:

```bash
--channelmod.modellist modellist_rfsimu_1 
```
O identificador `modellist_rfsimu_1` deve existir dentro do arquivo `channelmod_rfsimu.conf`.

**Passo 4.2:** Primeiro é preciso subir o core, utilizando os seguintes comandos:
```bash
cd /home/dco1020/oai-cn5g-fed/docker-compose

sudo python3 core-network.py --type start-basic --scenario 1
```

**Passo 4.3:** Em outro terminal suba a gNB com os seguintes comandos:
```bash

cd /home/dco1020/openairinterface5g/cmake_targets/ran_build/build

sudo RFSIMULATOR=server ./nr-softmodem --rfsim --sa -O ../../../targets/PROJECTS/GENERIC-NR-5GC/CONF/gnb.sa.band78.fr1.106PRB.usrpb210.conf --gNBs.[0].min_rxtxtime 6 --continuous-tx --rfsimulator.options chanmod --channelmod.modellist modellist_rfsimu_1 
```
**Passo 4.4:** Abra outro terminal e suba o UE com os seguintes comandos:

```bash
cd /home/dco1020/openairinterface5g/cmake_targets/ran_build/build

sudo RFSIMULATOR=192.168.70.129 ./nr-uesoftmodem -r 106 --numerology 1 --band 78 -C 3619200000 --rfsim --sa --nokrnmod -O ../../../targets/PROJECTS/GENERIC-NR-5GC/CONF/ue.conf --rfsimulator.options chanmod --channelmod.modellist modellist_rfsimu_1

```

### 5. Mudança de pathloss



Nesta seção é mostrado como alterar o parâmetro de perda de caminho (`pathloss`) e avaliar seus efeitos em métricas da rede, como throughput e RSRP.


**Passo 5.1:**  
Uma das formas de alterar o parâmetro é diretamente no arquivo `channelmod_rfsimu.conf`, modificando o valor da variável `ploss_dB`.

Para editar o arquivo, utilize o seguinte comando no terminal:

```bash
gedit /home/dco1020/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/channelmod_rfsimu.conf

```
![passo5_2](FIG/passo5_2.jpeg)
**Passo 5.2:** Outra forma de modificar esse parâmetro é diretamente via linha de comando, ao iniciar a gNB ou o UE. Basta adicionar:

```bash
--channelmod.modellist_rfsimu_1.[0].ploss_dB 30
```

 Onde:

- modellist_rfsimu_1 é o nome da lista definida no arquivo de configuração.
- [0] indica o índice do modelo a ser alterado dentro da lista.
- 30 é o valor da perda de caminho (em dB).
  
Exemplo para iniciar o UE com 30 dB de perda:

```bash
sudo RFSIMULATOR=192.168.70.129 ./nr-uesoftmodem -r 106 --numerology 1 --band 78 -C 3619200000 --rfsim --sa --nokrnmod -O ../../../targets/PROJECTS/GENERIC-NR-5GC/CONF/ue.conf --rfsimulator.options chanmod --channelmod.modellist modellist_rfsimu_1 --channelmod.modellist_rfsimu_1.[0].ploss_dB 30
```

**Passo 5.3:** Para visualizar os efeitos da mudança de ploss_dB, observe nos terminais os seguintes parâmetros:

RSRP (Received Signal Reference Power) nos logs do UE:

```bash
[NR_PHY] RSRP = -xx dBm
```
HARQ Stats (indicador de retransmissões no downlink):

```bash
Harq round stats for Downlink: xx/x/x
```
Throughput uplink, na gNB:
```bash
ulsch_total_bytes_received = <valor>
```

Ao mudar o pathloss do UE e fixar o da gNB em 0 dB tem-se os seguintes resultados:

#### **ploss_dB(UE) = 0 ploss_dB(gNB) = 0**

![passo5_3](FIG/ploss_ue0dB.jpeg)

#### **ploss_dB(UE) = 10 ploss_dB(gNB) = 0**

![passo5_31](FIG/ploss_ue10dB.jpeg)

#### **ploss_dB(UE) = 30 ploss_dB(gNB) = 0**

![passo5_32](FIG/ploss_ue30dB.jpeg)

### 6. Controle e Monitoramento via Telnet

###### O servidor Telnet do OAI permite controlar e ajustar parâmetros do modelo de canal em tempo real.

Telnet é um protocolo de rede para acessar remotamente outro dispositivo via linha de comando. O OAI permite acessar via telnet alguns softmoldems (executável que rodam entidades do OAI como a gNB e UE), por exemplo o nr-softmoldem (softmoldem da gNB)

Os tutorias de uso do telnet no OAI estão disponíveis [neste link](https://gitlab.eurecom.fr/oai/openairinterface5g/-/blob/develop/common/utils/telnetsrv/DOC/telnetsrv.md)

**Passo 6.1:** Habilitar o servidor Telnet

Adicione a seguinte opção na linha de comando de execução da gNB ou UE:

```bash
--telnetsrv porta
```
Exemplo:

```bash
sudo RFSIMULATOR=server ./nr-softmodem --rfsim --sa -O ../../../targets/PROJECTS/GENERIC-NR-5GC/CONF/gnb.sa.band78.fr1.106PRB.usrpb210.conf --gNBs.[0].min_rxtxtime 6 --continuous-tx --rfsimulator.options chanmod --telnetsrv 9090 --channelmod.modellist modellist_rfsimu_1 --channelmod.modellist_rfsimu_1.[0].ploss_dB 0
```

Selecione uma porta diferente para usar os dois servidores na mesma máquina.

Abra um terminal e adicione o comando abaixo:

```bash
telnet 127.0.0.1 9090
```

**Aperte Enter** para poder digitar os comandos via telnet.

<!-- ![passo5_31](FIG/passo6.jpeg) -->

para mudar o pathloss : 

```bash
channelmod modify 0 ploss 12
```
para mudar a potencia do ruido: 

```bash
channelmod modify 0 noise_power_dB 3
```
A figura baixo mostra a mensagem de respostas quando os parâmetros são modificados
![passo5_2](FIG/passo6_2.jpeg)

para sair do terminal do telnet use o comando:

```bash
Ctrl + 5
```
```bash
quit
```


**OBS:** Existem outras maneiras de mudar a potência do ruído

**passo 6.2 (opcional):** Uma opção é alterar o arquivo de configuração do modelo de canal, semelhante ao oque foi feito nos passos 1 da sessão 5.

Podemos fazer isso modificando o valor da variável `noise_power_dB` no arquivo `channelmod_rfsimu.conf`.

Para editar o arquivo, utilize o seguinte comando no terminal:

```bash
gedit /home/dco1020/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/channelmod_rfsimu.conf
```

Altere o valor da variável `noise_power_dB`

![passo5_2](FIG/passo6_1.jpeg)

**Passo 6.3 (opcional):** Outra opção é modificar esse parâmetro é diretamente via linha de comando, ao iniciar a gNB ou o UE. Basta adicionar:

```bash
--channelmod.modellist_rfsimu_1.[0].noise_power_dB 10
```


**OBS:** A configuração padrão do telnet no OAI só suporta acessar uma entidade por vez apenas na porta 9090. 

Portanto, caso queira acessar via telnet a gNB, basta subir a gNB com o comando do telnet e subir o UE sem o comando do telnet. Caso queira acesar o UE deve subir a gNB sem o comando do telnet e utilizar o comando apenas no UE para a porta 9090.


### 7. Monitoramento via nr-scope

Para verificar os efeitos das mudanças introduzidas pela simulação de canais, a ferramenta de constelação pode ser usada para rastrear e analisar os pontos da constelação de modulação. Esta ferramenta permite que os usuários visualizem o esquema de modulação que está sendo usado e avaliem a qualidade dos sinais recebidos. 

Ao observar os pontos da constelação, os usuários podem verificar se as alterações feitas na configuração do sistema resultaram em nr-scope

Este trecho do tutorial foi montado com base [neste material](https://openairinterface.org/wp-content/uploads/2022/06/OAI-Webinar-Series-Chapter-Four-Presentation-Slides.pdf)

#### Configuração 

**Passo 7.1:** Primeiramente para utilizar o nr-scope no OAI precisar instalar algumas dependências execute os comandos

```bash
sudo apt-get update
sudo apt-get install python3-tk python3-pip libuhd-dev uhd-host
```
#### Implantação com nr-scope 

Uma vez com as dependências necessárias, para realizar o deployment precisamos adicionar `-d`depois do `--rfsimulator.options chanmod` nas linhas de comando de execução utilizadas anteriormente 

**Passo 7.2:** Subir o core como realizado nos passos anteriores
```bash
cd /home/dco1020/oai-cn5g-fed/docker-compose
```
```bash
sudo python3 core-network.py --type stop-basic --scenario 1
```
```bash
sudo python3 core-network.py --type start-basic --scenario 1
```
**Passo 7.3:** Subir a gNB com o nr-scope 

entra na pasta

```bash
cd /home/dco1020/openairinterface5g/cmake_targets/ran_build/build
```
adiciona esse comando para subir a gNB

```bash
sudo RFSIMULATOR=server ./nr-softmodem --r
fsim --sa -O ../../../targets/PROJECTS/GENERIC-NR-5GC/CONF/gnb.sa.band78.fr1.106PRB.usrpb210.conf --gNBs.[0].min_rxtxtime 6 --continuous-tx --rfsimulator.options chanmod -d --telnetsrv --channelmod.modellist modellist_rfsimu_1
```


**Passo 7.3:** Subir o UE com o nr-scope
Abre outro terminal

entra na pasta

```bash
cd /home/dco1020/openairinterface5g/cmake_targets/ran_build/build
```
subir o UE

```bash
sudo RFSIMULATOR=192.168.70.129 ./nr-uesoftmod
em -r 106 --numerology 1 --band 78 -C 3619200000 --rfsim --sa --nokrnmod -O ../../../targets/PROJECTS/GENERIC-NR-5GC/CONF/ue.conf --rfsimulator.options chanmod -d --telnetsrv --channelmod.modellist modellist_rfsimu_1
```

A figura abaixo mostra o deployment de todas as partes com os valores padrão
![passo5_2](FIG/passo7.jpeg)


#### Visualização dos efeitos da potência do ruído via nr-scope

Ao subir a gNB ou UE com os comandos corretos altomaticamente abrirá o dashboard do nr-scope

O arquivo de configuração por padrão tem o valor da potência do ruido em 0dB, com isso temos os seguintes resultados

o dashboard da gNB tem o seguinte formato
![passo5_2](FIG/passo7_1.jpeg)

e o do UE é semelhante a esse
![passo5_2](FIG/passo7_2.jpeg)

Se subirmo o UE modificando o a potência do ruído para o valor de 10 dB como apresentado no **passo 6.3** e observarmos o dashboard do UE, vemos algo próximo ao apresentado na figura abaixo
![passo5_2](FIG/passo7_3.jpeg)

Podemos perceber no canto inferior direito da figura a constelação da recepção do UE mais dispersa em comparação a imagem anterior.

**OBS:** É possível durante a execução com o nr-scope acessar a gNB ou UE via telnet para modificar os parâmetros do canal e observar os efeitos em tempo real. Para isso revise os passos da sessão 6.

<!-- **Problemas Encontrados:** Por algum motivo não foi possível fazer o mesmo procedimento para observar os efeitos na gNB.  -->